In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

# **Algoritmo sequenziale:**
Implementazione algoritmo Greedy per la colorazione del grafo.

## **Funzionamento:**

*   Costruzione del grafo $\mathcal{G}(n, prob)$, dove $n$ è il numero di nodi e $prob$ la probabilità di estrazione di un arco.
*   Selezione di una permutazione casuale $σ$ sull' insieme $\{0 .. n\}.$
*   $∀i \in \{0 .. n\}$ il vertice $v_{σ_i}$ viene colorato con il primo colore disponibile.

## **Struttura:**

*   CPUColorer.cu → implementazione del colorer.
*   testerColorer.cu → costruzione del grafo e misurazione del tempo di esecuzione.



##**Risultati:**
Con un grafo $\mathcal{G}(10000, 1)$ si ottiene: 941.029419 ms

##**Problematica:**
Rispetto all'algoritmo implementato in parallelo risulta molto più veloce, quando il tempo di esecuzione dovrebbe essere $\mathcal{O}(n^2)$ nel caso di grafo completo.



In [15]:
%%cuda --name CPUColorer.cu

#include <iostream>
#include <cstdio>
#include <thrust/sequence.h>
#include <thrust/shuffle.h>
#include <thrust/random.h>
#include <thrust/count.h>

#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph_d.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph.h"
#include "/content/drive/MyDrive/graphcoloring/utils/common.h"

using namespace std;

void CPUcolorer(Coloring * col, GraphStruct *str){
	int n = str->nodeSize;

  int * perm = (int *) malloc(n * sizeof(int));
  thrust::sequence(perm, perm + n);
  thrust::default_random_engine g;
  thrust::shuffle(perm, perm + n, g);
 int counter = 0;

  for(int i = 0; i < n; i++){
    int currentNode = perm[i];
    uint offset = str->cumDegs[currentNode];
    uint deg = str->deg(currentNode);

    for (uint j = 0; j < deg; j++) {
        counter++;
      uint neighID = str->neighs[offset + j];
      int jColor = col->coloring[neighID];
      if (jColor != -1) col->usedColors[jColor] = true;

    }

    for(uint c = 0; c < n; c++){
      if(!col->usedColors[c]){
          col->coloring[currentNode] = c;
          break;
      }
    }
  }
}

Coloring* graphColoring(GraphStruct *str){
	int n = str->nodeSize;

	Coloring* col = (Coloring * ) malloc(sizeof(Coloring));

  col->coloring = (int *) malloc(n * sizeof(int));
	thrust::fill(col->coloring, col->coloring + n, -1);

  col->usedColors = (bool * ) malloc(n * sizeof(bool));
  thrust::fill(col->usedColors, col->usedColors + n, false);

  CPUcolorer(col, str);

	return col;
}

'File written in /content/src/CPUColorer.cu'

In [5]:
%%cuda --name testerColorer.cu

#include <iostream>
#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph.h"
#include "/content/drive/MyDrive/graphcoloring/utils/common.h"

#define THREADxBLOCK 128

// da fare: aggiungere stream? ad esempio per eseguire in modo concorrente stampa del grafo e la sua colorazione

int main(void) {
    unsigned int n = 10000;		 // number of nodes for random graphs
    float prob = 1;				    // density (percentage) for random graphs
    std::default_random_engine eng{0};  // fixed seed

    srand(time(0));
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // new graph with n nodes
    Graph graph(n, 1);

    // generate a random graph
    graph.randGraph(prob, eng);

    // get the graph struct
    GraphStruct *str = graph.getStruct();

    //print_d <<< 1, 1 >>> (str, true);
    //cudaDeviceSynchronize();

    cudaEventRecord(start);

    Coloring* col = graphColoring(str);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    //Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    printf("\n%f ms\n", milliseconds);

    int maxColor = 0;
    for(int i = 0; i < str->nodeSize; i++){
        if(maxColor < col->coloring[i]) maxColor = col->coloring[i];
    }
    printf("\nColore massimo: %d", maxColor+1);
    //printColoring(col, str, 1);

    return EXIT_SUCCESS;
}


'File written in /content/src/testerColorer.cu'

In [16]:
!nvcc -dc src/testerColorer.cu /content/src/CPUColorer.cu /content/drive/MyDrive/graphcoloring/graph/graph.cpp /content/drive/MyDrive/graphcoloring/graph/graph_d.cu
!nvcc testerColorer.o CPUColorer.o graph.o graph_d.o -o testerColorer
!./testerColorer


1259.225708 ms

Colore massimo: 10000

# **Algoritmo visto a lezione:**
Implementazione algoritmo Luby per la colorazione del grafo.

##**Risultati:**
Con un grafo $\mathcal{G}(10000, 1)$ si ottiene: 26493.421875 ms

##**Problematica:**
Rispetto all'algoritmo implementato in maniera sequenziale risulta molto più lento e soprattutto trova come coloratura massima 9760, inferiore al numero di nodi.


In [9]:
%%cuda --name coloring.h
#pragma once

#include <curand_kernel.h>
#include "/content/drive/MyDrive/graphcoloring/graph/graph.h"
#include "/content/drive/MyDrive/graphcoloring/utils/common.h"

/**
 *  graph coloring struct (colors are: 1,2,3,..,k)
 */

struct Coloring {
	bool		uncoloredNodes;
	uint		numOfColors;
	uint	*	coloring;   // each element denotes a color
};

Coloring* LubyGreedy(GraphStruct*);
void printColoring (Coloring*, GraphStruct*, bool);
__global__ void LubyJPcolorer (Coloring*, GraphStruct*, uint*) ;
__global__ void init(uint seed, curandState_t*, uint*, uint);
__global__ void findIS (Coloring*, GraphStruct*, uint*);
__global__ void print_d(GraphStruct*, bool);

'File written in /content/src/coloring.h'

In [10]:
%%cuda --name Luby.cu


#include "coloring.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph_d.h"
#include "/content/drive/MyDrive/graphcoloring/utils/common.h"

using namespace std;

#define THREADxBLOCK 128

Coloring* LubyGreedy(GraphStruct *str) {
	// set coloring struct

	Coloring* col;
	gpuErrchk(cudaMallocManaged(&col, sizeof(Coloring)));
	uint n = str->nodeSize;
	col->uncoloredNodes = true;

	// cudaMalloc for arrays of struct Coloring
	gpuErrchk(cudaMallocManaged( &(col->coloring), n * sizeof(uint)));
	memset(col->coloring,0,n);

	// allocate space on the GPU for the random states
	curandState_t* states;
	uint* weigths;
	cudaMalloc((void**) &states, n * sizeof(curandState_t));
	cudaMalloc((void**) &weigths, n * sizeof(uint));
	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );
	uint seed = 0;
	init <<< blocks, threads >>> (seed, states, weigths, n);

	// start coloring (dyn. parall.)
	//LubyJPcolorer <<< 1, 1 >>> (col, str, weigths);

//#####################
	// loop on CPU

	// loop on ISs covering the graph
	col->numOfColors = 0;
	while (col->uncoloredNodes) {
		col->uncoloredNodes = false;
		col->numOfColors++;
		findIS <<< blocks, threads >>> (col, str, weigths);
		cudaDeviceSynchronize();
	}
//#####################


	cudaFree(states);
	cudaFree(weigths);
	return col;
}

/**
 * find an IS
 */
__global__ void findIS (Coloring* col, GraphStruct *str, uint* weights) {
	uint idx = threadIdx.x + blockDim.x * blockIdx.x;

	if (idx >= str->nodeSize)
		return;

	if (col->coloring[idx])
		return;

	uint offset = str->cumDegs[idx];
	uint deg = str->cumDegs[idx + 1] - str->cumDegs[idx];

	bool candidate = true;
	for (uint j = 0; j < deg; j++) {
		uint neighID = str->neighs[offset + j];
		if (!col->coloring[neighID] &&
				((weights[idx] < weights[neighID]) ||
				((weights[idx] == weights[neighID]) && idx < neighID))) {
			candidate = false;
		}
	}
	if (candidate) {
		col->coloring[idx] = col->numOfColors;
	}
	else
		col->uncoloredNodes = true;
}


/**
 *  this GPU kernel takes an array of states, and an array of ints, and puts a random int into each
 */
__global__ void init (uint seed, curandState_t* states, uint* numbers, uint n) {
	uint idx = blockIdx.x * blockDim.x + threadIdx.x;
	if (idx > n)
			return;
	curand_init(seed, idx, 0, &states[idx]);
	numbers[idx] = curand(&states[idx])%n*n;
}


/**
 * Luby IS & Jones−Plassmann colorer
 */
__global__ void LubyJPcolorer (Coloring* col, GraphStruct *str, uint* weights) {
	dim3 threads (THREADxBLOCK);
	dim3 blocks ((str->nodeSize + threads.x - 1) / threads.x, 1, 1 );

	// loop on ISs covering the graph
	col->numOfColors = 0;
	while (col->uncoloredNodes) {
		col->uncoloredNodes = false;
		col->numOfColors++;
		findIS <<< blocks, threads >>> (col, str, weights);
		//cudaDeviceSynchronize();
	}
}



'File written in /content/src/Luby.cu'

In [11]:
%%cuda --name test_Luby.cu

#include "coloring.h"

int main(void) {
	unsigned int n = 10000;		 // number of nodes for random graphs
	float prob = 1;				    // density (percentage) for random graphs
	std::default_random_engine eng{0};  // fixed seed

  srand(time(0));
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

	// new graph with n nodes
	Graph graph(n,1);

	// generate a random graph
	graph.randGraph(prob,eng);

	// get the graph struct
	GraphStruct *str = graph.getStruct();

  //print_d <<< 1, 1 >>> (str, true);

  cudaEventRecord(start);

  Coloring* col = LubyGreedy(str);
  cudaDeviceSynchronize();

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  //Stampo in millisecondi quanto tempo ci ha messo a colorare il grafo.
  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  printf("%f ms\n", milliseconds);

  int maxColor = 0;
  for(int i = 0; i < str->nodeSize; i++){
      if(maxColor < col->coloring[i]) maxColor = col->coloring[i];
  }
  printf("\nColore massimo: %d", maxColor+1);
  //printColoring(col, str, 1);

	return EXIT_SUCCESS;
}

'File written in /content/src/test_Luby.cu'

In [12]:
!nvcc -dc src/test_Luby.cu src/Luby.cu /content/drive/MyDrive/graphcoloring/graph/graph.cpp /content/drive/MyDrive/graphcoloring/graph/graph_d.cu
!nvcc test_Luby.o Luby.o graph.o graph_d.o -o testLuby
!./testLuby

26827.693359 ms

Colore massimo: 9745

# **Algoritmo Largest Degree First (LDF):**
Implementazione algoritmo Largest Degree First (LDF) per la colorazione del grafo.

# **Funzionamento:**
*   Costruzione del grafo $\mathcal{G}(n, prob)$, dove $n$ è il numero di nodi e $prob$ la probabilità di estrazione di un arco.
*   Assegnamento casuale dei pesi $w$ sull' insieme $\{0 .. n\}.$
*   (In parallelo) $∀i \in \{0 .. n\}$ il vertice $v_i$ viene selezionato come candidato se risulta essere un massimo locale utilizzado come metrica i gradi (nel caso di ugualianza si confrontano i pesi).
*   (In parallelo con stream?) Colorazione del candidato trovato al punto precedente con il primo colore disponibile.

# **Struttura:**
*   LDFColorer.cu → implementazione del colorer.
*   testerColorer.cu → costruzione del grafo e misurazione del tempo di esecuzione.

##**Risultati:**
Con un grafo $\mathcal{G}(10000, 1)$ si ottiene: 253779.406250 ms

##**Problematica:**
Rispetto all'algoritmo implementato in maniera sequenziale risulta MOLTO più lento.


In [13]:

%%cuda --name LDFColorer.cu

#include <cuda.h>
#include <iostream>
#include <curand_kernel.h>
#include <thrust/sequence.h>
#include <thrust/shuffle.h>
#include <thrust/random.h>
#include <thrust/count.h>

#include "/content/drive/MyDrive/graphcoloring/graph/coloring.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph.h"
#include "/content/drive/MyDrive/graphcoloring/graph/graph_d.h"
#include "/content/drive/MyDrive/graphcoloring/utils/common.h"

#define THREADxBLOCK 128
#define NSTREAM 4

using namespace std;

__global__ void findCandidate(Coloring* col, GraphStruct *str, bool * candidate){
    int n = str->nodeSize;
	  uint i = threadIdx.x + blockDim.x * blockIdx.x;
		bool flag = true; // vera sse il nodo ha peso locale massimo

    if (i >= n) return;

		// ignora i nodi già colorati
		if (col->coloring[i] != -1) return;

		int iWeight = str->weights[i];

		// guarda i pesi del vicinato
		uint offset = str->cumDegs[i];
		uint deg = str->cumDegs[i + 1] - str->cumDegs[i];

		for (uint j = 0; j < deg; j++) {
			uint neighID = str->neighs[offset + j];
      uint neighDeg = str->cumDegs[neighID + 1] - str->cumDegs[neighID];
			// ignora i vicini già colorati (e te stesso)
			int jColor = col->coloring[neighID];

      if(jColor != -1){
          col->usedColors[n * i + jColor] = true;
          continue;
      }
      if(i == neighID) continue;

			int jWeight = str->weights[neighID];

			if (deg < neighDeg){
          flag = false;
      }
      else if (deg == neighDeg){
          if(iWeight < jWeight){
              flag = false;
          }
      }


  }
    if(flag){
        candidate[i] = true;
      }
}

__global__ void colorer (Coloring* col, GraphStruct *str, bool* candidate, int offset) {
  uint i = threadIdx.x + blockDim.x * blockIdx.x;
  int n = str->nodeSize;

  if(candidate[i]){
        int color = 0;
        while (col->usedColors[n * (i+offset) + color]) color++;

        // Assegna il primo colore libero al nodo corrente
        col->coloring[i + offset] = color;
    }
}

Coloring* graphColoring(GraphStruct *str) {
	int n = str->nodeSize;
    int r = rand();



	Coloring* col;
	gpuErrchk(cudaMallocManaged(&col, sizeof(Coloring)));

    gpuErrchk(cudaMallocManaged(&(col->coloring), n * sizeof(int)));
	thrust::fill(col->coloring, col->coloring + n, -1);


    gpuErrchk(cudaMallocManaged(&(col->usedColors), n * n * sizeof(bool)));
    thrust::fill(col->usedColors, col->usedColors + (n * n), false);

    bool* candidate;
    gpuErrchk(cudaMallocManaged(&(candidate), n * sizeof(bool)));
    thrust::fill(candidate, candidate + n, false);

    // Generazione pesi
    thrust::sequence(str->weights, str->weights + n);
    thrust::default_random_engine g;
    thrust::shuffle(str->weights, str->weights + n, g);

 int iElem = n / NSTREAM;
	dim3 threads ( THREADxBLOCK);
	dim3 blocks ((n + threads.x - 1) / threads.x, 1, 1 );
  dim3 blocksS ((iElem + threads.x - 1) / threads.x, 1, 1);



    cudaStream_t stream[NSTREAM];

    for (int i = 0; i < NSTREAM; ++i)
      gpuErrchk(cudaStreamCreate(&stream[i]));

	for(int c = 0; c < n; c++){
    findCandidate<<<blocks, threads>>>(col, str, candidate);
    gpuErrchk(cudaPeekAtLastError());
		gpuErrchk(cudaDeviceSynchronize());
    for(int i=0; i < NSTREAM; ++i){
        int ioffset = i * iElem;
        colorer<<<blocksS, threads, 0, stream[i]>>>(col, str, &candidate[ioffset], ioffset);
    }
    gpuErrchk(cudaPeekAtLastError());
		gpuErrchk(cudaDeviceSynchronize());
    thrust::fill(candidate, candidate + n, false);



        int left = (int)thrust::count(col->coloring, col->coloring + n ,-1);
        if (left == 0){
            break;
        }
	}

  for (int i = 0; i < NSTREAM; ++i)
      gpuErrchk(cudaStreamDestroy(stream[i]));

    return col;
}

'File written in /content/src/LDFColorer.cu'

In [14]:
!nvcc -dc src/testerColorer.cu /content/src/LDFColorer.cu /content/drive/MyDrive/graphcoloring/graph/graph.cpp /content/drive/MyDrive/graphcoloring/graph/graph_d.cu
!nvcc testerColorer.o LDFColorer.o graph.o graph_d.o -o testerColorer
!./testerColorer


255170.187500 ms

Colore massimo: 10000